In [1]:
#-------------------------------------IMPORTS-------------------------------------#
import torch
import gzip
import numpy as np
from random import randint
import torch.utils.data as data_utils
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics.pairwise import cosine_similarity
from numpy import linalg as LA

In [2]:
#-------------------------------------GLOBAL VARIABLES-------------------------------------#
NUM_TRAINING_EXAMPLES = 22850 #FOR DATA BATCHER, WHEN DEPLOYED SHOULD BE ALL TRAINING EXAMPLES
PARAMETER_MAX_TITLE_LENGTH = 38
NUM_FEATURES_PER_WORD = 200 #DO NOT CHANGE. FIXED at 200

In [3]:
#-------------------------------------HELPER FUNCTIONS-------------------------------------#
NUM_NEGATIVE_SAMPLES=20

def N_random_values_in_list(full_list, N):
    x=0
    lower_bound  = 0
    upper_bound = len(full_list)-1
    sample_list=[]
    random_nums=[]
    while x < min(N,len(full_list)):
        random_num = randint(lower_bound, upper_bound) # inclusive range
        if random_num in random_nums:
            continue
        else:
            random_nums.append(random_num)
            x += 1
    return [full_list[i] for i in random_nums]

def convert_to_list(filename):
    if filename.endswith('gz'):
        with gzip.open(filename,'r')as f:
            text_tokens = f.readlines()
    else:
        with open(filename, 'r') as f:
            text_tokens = f.readlines()
    text_tokens = [token.replace('\n','').split('\t') for token in text_tokens]
    text_tokens = [[token[0], token[1].split(' '), token[2].split(' ')] for token in text_tokens]
    #print text_tokens[0]
    return text_tokens

#Sample:question_id, similar_question_id, negative_question_id
def convert_to_samples(filename, is_dev_or_test = False):
    my_list=convert_to_list(filename)
    new_samples=[]
    for original_sample in my_list:
        if is_dev_or_test:
            new_samples.append([original_sample[0], similar, negative_samples])
        else:
            for similar in original_sample[1]:
                random_negative_samples = N_random_values_in_list(original_sample[2],NUM_NEGATIVE_SAMPLES)
                new_samples.append([original_sample[0], similar, random_negative_samples])# change this to include all negative 
                                                                                     # examples later
    return new_samples
def make_lookup_table_for_training_data(filename):
    lookup={}
    text_token_list=convert_to_list(filename)
    for token in text_token_list:
        lookup[token[0]] = {'title':token[1],'question':token[2]}
    return lookup
        
#takes  sample_ids of [[q1,p1,n1],[q2,p2,n2]....]
#outputs titles like [[q1_title, p1_title, n1_title],[q2_title,p2_title,n2_title]...]
def convert_sampleids_to_titles(sample_ids,lookup, is_dev_or_test = False):
    #each sample_id [question_id, pos_id, [neg_ids]]
    #print type(sample_ids)==list, "first"
   
    titles = []
   # print "sample_ids", sample_ids
    for sample_id in sample_ids:
        if is_dev_or_test:
            #omit similar questions, only needed for evaluation
            sample_id=[sample_id[0]]+sample_id[2][:]
         #flatten list: [question_id, pos_id, [neg_ids]] --> [question_id, pos_id, neg_id1, neg_id2, ...]
        else:
            sample_id= sample_id[:2]+sample_id[2][:]
        #sample_id : question_id, similar_question_id, negative_question_id
        #try:
        current=[]
        for identity in sample_id:
            try:
                current.append(lookup[str(identity)]['title'])
            except:
                print identity
        titles.append(current)
           # print type(sample_id)==list
        #except:
        #    print sample_id, "is sample id", type(sample_id)==list
    return titles
def remove_non_ascii(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

def extract_features(word):
    try:
        word=remove_non_ascii(word)
        word=word.encode('utf-8')
    except:
        print(word)
    return word_to_vec.get(word,[0.0 for i in range(200)])

def find_maximum_title_and_body_length(lookup_table):
    max_len_title = -1
    max_len_question = -1
    max_len_question_id = 0
    for key, dict_val in lookup_table.iteritems():
        len_title = len(dict_val['title'])
        len_question = len(dict_val['question'])
        if len_title > max_len_title:
             max_len_title = len_title
        if len_question > max_len_question:
            max_len_question = len_question
            max_len_question_id = key
    return max_len_title, max_len_question

def title_to_feature_matrix(title_word_list):
    feature_matrix = []
    for idx, word in enumerate(title_word_list):
        if idx == PARAMETER_MAX_TITLE_LENGTH:
            break
        else:
            word_features = extract_features(word)
            feature_matrix.append(word_features)
        
    #Pad the feature with zeros to ensure all inputs to the net have the same dimension
    feature_matrix += [[0] * NUM_FEATURES_PER_WORD] * (PARAMETER_MAX_TITLE_LENGTH - len(title_word_list))
    #print np.array(feature_matrix).T.shape
    return np.array(feature_matrix).T



#array is structured like a batch of features 50x200x38
def find_start_of_padding_for_batch(batch):
    vec_lengths_in_batch = []
    for batch_num in range(0, len(batch)):
        single_vec = batch[batch_num]
        length = find_start_of_padding_single_vec(single_vec) + 1
        vec_lengths_in_batch.append(length)
    return vec_lengths_in_batch

#batch = 200x38
def find_start_of_padding_single_vec(single_vec):
    for idx in range(len(single_vec[0])-1, -1, -1):
        if single_vec[0][idx] != 0.:
            return idx
    #if the whole sequence is 0s
    return 0
def create_mask(word_length):
    return np.array([[1. / word_length] * HIDDEN_SIZE] * word_length + [[0] * HIDDEN_SIZE] * (MAX_TITLE_LENGTH - word_length)).T
#-------------------------------------CREATE DATA BATCHER-------------------------------------#
# where samples[0] = 1 (target) + 1 (positive) + n (negative)
# 
def create_data_loader(samples, shuffle_data = True, is_test_or_dev = False):
    features = []
    for sample in samples:
        target_title = sample[0]
        positive_title = sample[1]
        negative_titles = sample[2:]

        target_features = title_to_feature_matrix(target_title)
        positive_features = title_to_feature_matrix(positive_title)
        if len(negative_titles)==0:
            all_features=[target_features, positive_features]
        else:
            n_negative_features = [title_to_feature_matrix(negative_title) for negative_title in negative_titles]

            all_features = [target_features, positive_features] + n_negative_features
        #print all_features, "is all"
       # if is_test_or_dev:
       #     all_features = [target_features,]
        features.append(all_features)

            
    targets = torch.LongTensor(len(features)).zero_()
    dataset = data_utils.TensorDataset(torch.FloatTensor(features), targets)
    data_loader = data_utils.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = shuffle_data)
    return data_loader

def create_dataset(samples, shuffle_data = True, is_test_or_dev = False):
    features = []
    for sample in samples:
        target_title = sample[0]
        positive_title = sample[1]
        negative_titles = sample[2:]

        target_features = title_to_feature_matrix(target_title)
        positive_features = title_to_feature_matrix(positive_title)
        if len(negative_titles)==0:
            all_features=[target_features, positive_features]
        else:
            n_negative_features = [title_to_feature_matrix(negative_title) for negative_title in negative_titles]

            all_features = [target_features, positive_features] + n_negative_features
        #print all_features, "is all"
       # if is_test_or_dev:
       #     all_features = [target_features,]
        features.append(all_features)

            
    targets = torch.LongTensor(len(features)).zero_()
    dataset = data_utils.TensorDataset(torch.FloatTensor(features), targets)
    #data_loader = data_utils.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = shuffle_data)
    return dataset

In [10]:
#-------------------------------------NET GLOBAL VARIABLES-------------------------------------#
#text_tokenized.txt.gz has id \t title \t question body
text_tokenized='askubuntu/text_tokenized.txt.gz'
lookup = make_lookup_table_for_training_data(text_tokenized)
MAX_TITLE_LENGTH, MAX_BODY_LENGTH = find_maximum_title_and_body_length(lookup)

In [14]:
KERNEL_SIZE = 3 #MAKE SURE THIS NUMBER IS ODD SO THAT THE PADDING MAKES SENSE
PADDING = (KERNEL_SIZE - 1) / 2
INPUT_SIZE = 200
HIDDEN_SIZE = 600
LEARNING_RATE = 1e-5
MARGIN = 0.2
NUM_EPOCHS = 10
BATCH_SIZE = 50
NUM_BATCHES = NUM_TRAINING_EXAMPLES/BATCH_SIZE

In [12]:
#-------------------------------------LOAD DATA-------------------------------------#

word_embeddings = 'askubuntu/vector/vectors_pruned.200.txt.gz'
f = gzip.open(word_embeddings, 'r')
wv_text = [ ]
lines = f.readlines()
for line in lines:
    wv_text.append(line.strip())

word_to_vec = {}

for line in wv_text:
    parts = line.split()
    word = parts[0]
    vector = np.array([float(v) for v in parts[1:]])
    word_to_vec[word] = vector
f.close()


In [32]:
#train_random.txt
#(1) the query question ID, (2) the list of similar question IDs, and (3) the list of randomly selected question IDs.

NUM_TRAINING_EXAMPLES=2000

train_random_filename='askubuntu/train_random.txt'

#Each line contains (1) the query question ID, (2) the list of similar question IDs, (3) the list of 20 candidate question IDs and (4) the associated BM25 scores of these questions computed by the Lucene search engine. The second field (the set of similar questions) is a subset of the third field.
dev_filename='askubuntu/dev.txt'
test_filename='askubuntu/test.txt'

train_samples = convert_to_samples(train_random_filename)
#dev_samples = convert_to_samples(dev_filename, is_dev_or_test = True)
#test_samples = convert_to_samples(test_filename, is_dev_or_test = True)
print "training"
train_titles_only = convert_sampleids_to_titles(train_samples, lookup)

train_data_loaders=[]
for i in range(1000,NUM_TRAINING_EXAMPLES,50):
    train_data_loader = create_dataset(train_titles_only[i:i+50])
    print "made training data loader", i
    train_data_loaders.append(train_data_loader)
final_train_dataset=data_utils.ConcatDataset(train_data_loaders)
    #print "dev"
#dev_titles_only = convert_sampleids_to_titles(dev_samples, lookup)
#dev_data_loader = create_data_loader(dev_titles_only)
#print "test"
#test_titles_only = convert_sampleids_to_titles(test_samples, lookup)
#test_data_loader = create_data_loader(test_titles_only, shuffle_data = False)



 training
made training data loader 1000
made training data loader 1050
made training data loader 1100
made training data loader 1150
made training data loader 1200
made training data loader 1250
made training data loader 1300
made training data loader 1350
made training data loader 1400
made training data loader 1450
made training data loader 1500
made training data loader 1550
made training data loader 1600
made training data loader 1650
made training data loader 1700
made training data loader 1750
made training data loader 1800
made training data loader 1850
made training data loader 1900
made training data loader 1950


In [33]:
final_train_data_loader=data_utils.DataLoader(final_train_dataset, batch_size = BATCH_SIZE, shuffle = True)

In [6]:
#-------------------------------------CNN-------------------------------------#
class CNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(200, hidden_dim, KERNEL_SIZE, padding = PADDING),
#             nn.ReLU()
            nn.Tanh()
        )
    def forward(self, x):
        x = self.layer1(x)
        return x

In [7]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)


def evaluate_model_on_test(model, test_data_filename):
    test_data_loader =convert_to_list(test_data_filename)
    
    model.eval() #change model to 'eval' mode
    correct = 0
    total = 0
    p5 = 0
    MRR = 0
    all_data = []
    for sample in test_data_loader:
        #print sample
        target_id=sample[0]
        similar_ids=sample[1]
        #print similar_ids, 'similar'
        candidate_ids=sample[2]
        #print candidate_ids, 'candidates'
        titles = convert_sampleids_to_titles([sample], lookup,is_dev_or_test=True)[0]

        #print titles
        twenty_one_features = [title_to_feature_matrix(title) for title in titles]
        
        cosine_sims = sample_to_cosine_sims(twenty_one_features, model = model)
        #print cosine_sims
        #_, predicted = torch.max(cosine_sims, 0)
       # total += label.size(0)
       # label = label.squeeze()
        
        #correct += ((predicted == label).sum().data.numpy().squeeze())
        cosine_sims = [cos_sim.data.numpy()[0] for cos_sim in cosine_sims]
        #cosine_sims = cosine_sims.data.numpy().T
        #print cosine_sims, "is cos sims"
        #print candidate_ids, 'is candidates'
        gold_labels = [1 if identity in similar_ids else 0 for identity in candidate_ids]
        #print gold_labels, "is gold"
        
        index_order = sorted(range(len(cosine_sims)), reverse = True,key=lambda k: cosine_sims[k])
        #print index_order, "is index order"
        
        data = [gold_labels[index] for index in index_order]
        all_data.append(data)
#         for instance_idx in range(len(cosine_sims)):
#             cosine_sims_at_idx = cosine_sims[instance_idx]            
#             print "Cosine sims at idx: {}".format(cosine_sims_at_idx)
#             a
#             loc = np.squeeze(np.where(cosine_sims_at_idx.argsort()[::-1] == 0))
#             data_instance = [0] * len(cosine_sims_at_idx)
#             data_instance[loc] = 1
#             data.append(data_instance)
        
#         for instance_idx in range(len(cosine_sims)):
#             cosine_sims_at_idx = cosine_sims[instance_idx]
#             top_5_indices = cosine_sims_at_idx.argsort()[-5:][::-1]
#             label_at_idx = 0
#             if label_at_idx in top_5_indices:
#                 p5 += 1
            
#             MRR += 1. / (1. + np.squeeze(np.where(cosine_sims_at_idx.argsort()[::-1] == 0)))
    return all_data 

def sample_to_cosine_sims(sample, model = None):
    #RE-ORDER DIMENSIONS OF THE SAMPLE
    sample = Variable(torch.FloatTensor(sample))
    #print sample.unsqueeze(0)
    # 1x21x200x38
    sample = sample.unsqueeze(0).permute(1, 0, 2, 3)
    if model != None:
        net = model
    #21 x 1 x 200 x 38
    target_question_features = sample[0] # 1 x 1 x 200 x 38
   # print target_question_features.data.numpy()shape, 'Is target"'
    other_question_features = sample[1:] # 20 x 1 x 200 x 38
   # print other_question_features.data.numpy()shape, "is other"

    #Determine lengths to know how many vectors to take the average across.
    target_question_lengths = find_start_of_padding_for_batch(target_question_features.data)
    other_questions_lengths = [find_start_of_padding_for_batch(negative.data) for negative in other_question_features]

    #RUN THROUGH NET
    target_question_net_output = net(target_question_features)
    other_question_net_outputs = [net(negative) for negative in other_question_features]

    #CREATE MASKS
    target_questions_masks = [create_mask(_) for _ in target_question_lengths] #DIM = 50 x 100 x 38
    other_questions_masks = [[create_mask(length) for length in length_list] #DIM = 50 x 20 x 100 x 38
                                  for length_list in other_questions_lengths]

    #APPLY MASKS
    #Should the multiplicands, the masks, be Float Tensors or Variables? May have to be float tensors to ensure
    #    pytorch's directed graph back-prop is maintained.

    target_question_net_output_masked = target_question_net_output * Variable(torch.FloatTensor(target_questions_masks))
    other_questions_net_output_masked = [other_question_net_outputs[idx] * 
                                              Variable(torch.FloatTensor(other_questions_masks[idx]))
                                              for idx in range(20)]

    #SUM OVER WORDS
    target_question_net_output_masked_summed = torch.sum(target_question_net_output_masked, dim = 2) #DIM = 50 x 100
    other_questions_net_output_masked_summed = [torch.sum(
                                                    other_questions_net_output_masked[idx], dim = 2
                                                    )for idx in range(20)] #DIM = 20 x 50 x 100


    #cosine_similarity_pos = cos(target_question_net_output_masked_summed, positive_question_net_output_masked_summed)
    # ^ DIM = 50
    cosine_similarities = [cos(target_question_net_output_masked_summed, other_questions_net_output_masked_summed[idx])
                              for idx in range(20)]
    # ^ DIM = 20 x 50
    #

    #cosine_similarities = torch.stack([cosine_similarity_pos] + cosine_similarities_neg) # DIM = 21 x 50
    return cosine_similarities


In [37]:
LEARNING_RATE=1e-6

In [38]:
INPUT_DIM = (MAX_TITLE_LENGTH, NUM_FEATURES_PER_WORD)
#net = CNN(INPUT_SIZE, HIDDEN_SIZE)
net=torch.load("Net_Epoch_1e5_1")
criterion = nn.MultiMarginLoss(p=1, margin=MARGIN, weight = None, size_average=True) #HAHA just put these in to look smart 
optimizer = torch.optim.Adam(net.parameters(), lr = LEARNING_RATE)

cos = nn.CosineSimilarity(dim=1, eps=1e-6)
print "Beginning training..."
# ----TRAINING
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for idx,(sample, label) in enumerate(final_train_data_loader):
        print idx,
#     for idx in range(NUM_BATCHES):
        #Sample shape: [50, 22, 200, 38]
        #50 - Batch size, 22 - Num questions per data point
        sample = Variable(sample, requires_grad = True)
        label = Variable(label, requires_grad = True)

        #RE-ORDER DIMENSIONS OF THE SAMPLE
        sample = sample.permute(1, 0, 2, 3)
        
        target_question_features = sample[0] # 50 x 200 x 38
        positive_question_features = sample[1] # 50 x 200 x 38
        N_negative_question_features = sample[2:] #20 x 50 x 200 x 38
        
        #Determine lengths to know how many vectors to take the average across.
        target_question_lengths = find_start_of_padding_for_batch(target_question_features.data)
        positive_question_lengths = find_start_of_padding_for_batch(positive_question_features.data)
        N_negative_questions_lengths = [find_start_of_padding_for_batch(negative.data) for negative in N_negative_question_features]
        
        #RUN THROUGH NET
        target_question_net_output = net(target_question_features)
        positive_question_net_output = net(positive_question_features)
        N_negative_question_net_outputs = [net(negative) for negative in N_negative_question_features]
        
        #CREATE MASKS
        target_questions_masks = [create_mask(_) for _ in target_question_lengths] #DIM = 50 x 100 x 38
        positive_questions_masks = [create_mask(_) for _ in positive_question_lengths] #DIM = 50 x 100 x 38
        N_negative_questions_masks = [[create_mask(length) for length in length_list] #DIM = 50 x 20 x 100 x 38
                                      for length_list in N_negative_questions_lengths]
        
        #APPLY MASKS
        #Should the multiplicands, the masks, be Float Tensors or Variables? May have to be float tensors to ensure
        #    pytorch's directed graph back-prop is maintained.

        target_question_net_output_masked = target_question_net_output * Variable(torch.FloatTensor(target_questions_masks))
        positive_question_net_output_masked = positive_question_net_output * Variable(torch.FloatTensor(positive_questions_masks))
        N_negative_questions_net_output_masked = [N_negative_question_net_outputs[idx] * 
                                                  Variable(torch.FloatTensor(N_negative_questions_masks[idx]))
                                                  for idx in range(NUM_NEGATIVE_SAMPLES)]

        #SUM OVER WORDS
        target_question_net_output_masked_summed = torch.sum(target_question_net_output_masked, dim = 2) #DIM = 50 x 100
        positive_question_net_output_masked_summed = torch.sum(positive_question_net_output_masked, dim = 2) #DIM = 50 x 100
        N_negative_questions_net_output_masked_summed = [torch.sum(
                                                        N_negative_questions_net_output_masked[idx], dim = 2
                                                        )for idx in range(NUM_NEGATIVE_SAMPLES)] #DIM = 20 x 50 x 100

        
        cosine_similarity_pos = cos(target_question_net_output_masked_summed, positive_question_net_output_masked_summed)
        # ^ DIM = 50
        cosine_similarities_neg = [cos(target_question_net_output_masked_summed, N_negative_questions_net_output_masked_summed[idx])
                                  for idx in range(NUM_NEGATIVE_SAMPLES)]
        # ^ DIM = 20 x 50
        #

        cosine_similarities = torch.stack([cosine_similarity_pos] + cosine_similarities_neg) # DIM = 21 x 50
        
        optimizer.zero_grad()
        
        cosine_similarities = torch.t(cosine_similarities)
        label = torch.squeeze(label)
        loss = criterion(cosine_similarities, label)
        
        loss.backward()
        
        running_loss += loss.data[0]
        
        optimizer.step()
        
    torch.save(net, "Net_Epoch_1e5_{}".format(epoch))
    print ""
    print "Loss after epoch " + str(epoch) + " :" + str(running_loss)
    data = evaluate_model_on_test(net, 'askubuntu/dev.txt')
    eval = Evaluation(data)
    print "MAP: ",eval.MAP()
    print "MRR: ", eval.MRR()
    print "P@1: ",eval.Precision(1)
    print "P@5: ",eval.Precision(5)
    print "test"
    data = evaluate_model_on_test(net, 'askubuntu/test.txt')
    eval = Evaluation(data)
    print "MAP: ",eval.MAP()
    print "MRR: ", eval.MRR()
    print "P@1: ",eval.Precision(1)
    print "P@5: ",eval.Precision(5)
    
         


Beginning training...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
Loss after epoch 0 :0.196269263048
MAP:  0.538974168588
MRR:  0.667962543733
P@1:  0.539682539683
P@5:  0.431746031746
test
MAP:  0.548128579171
MRR:  0.684542323676
P@1:  0.55376344086
P@5:  0.41935483871
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
Loss after epoch 1 :0.194819496013
MAP:  0.538923234318
MRR:  0.667981995958
P@1:  0.539682539683
P@5:  0.430687830688
test
MAP:  0.547918275735
MRR:  0.683646266329
P@1:  0.55376344086
P@5:  0.41935483871
0 1 2 3

KeyboardInterrupt: 

In [8]:
class Evaluation():

    def __init__(self,data):

        self.data = data


    def Precision(self,precision_at):
        scores = []
        for item in self.data:
            temp = item[:precision_at]
            if any(val==1 for val in item):
                scores.append(sum([1 if val==1 else 0 for val in temp])*1.0 / len(temp) if len(temp) > 0 else 0.0)
        return sum(scores)/len(scores) if len(scores) > 0 else 0.0


    def MAP(self):
        scores = []
        missing_MAP = 0
        for item in self.data:
            temp = []
            count = 0.0
            for i,val in enumerate(item):
                if val == 1:
                    count += 1.0
                    temp.append(count/(i+1))
            if len(temp) > 0:
                scores.append(sum(temp) / len(temp))
            else:
                missing_MAP += 1
        return sum(scores)/len(scores) if len(scores) > 0 else 0.0


    def MRR(self):

        scores = []
        for item in self.data:
            for i,val in enumerate(item):
                if val == 1:
                    scores.append(1.0/(i+1))
                    break

        return sum(scores)/len(scores) if len(scores) > 0 else 0.0

In [ ]:
model = torch.load("Net_Epoch_5")
data = evaluate_model_on_test(model, 'askubuntu/dev.txt')


In [24]:
eval = Evaluation(data)
print "MAP: ",eval.MAP()
print "MRR: ", eval.MRR()
print "P@1: ",eval.Precision(1)
print "P@5: ",eval.Precision(5)

MAP:  0.551681464457
MRR:  0.687877506776
P@1:  0.559139784946
P@5:  0.412903225806


In [8]:
LAMBDA = 0.5
#-------------------------------DOMAIN ADAPTATION----------------------------------#
class DCNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(200, hidden_dim, KERNEL_SIZE, padding = PADDING),
#             nn.ReLU()
            nn.Tanh()
        )
        self.layer2 = 
    def forward(self, x):
        x = self.layer1(x)
        return x

In [9]:
#-------------------------------DOMAIN ADAPTATION (MAY BE UNNECESSARY) ----------------------------------#
class GradReverse(torch.autograd.Function):
    def forward(self, x):
        return x.view_as(x)
    def backward(self, grad_output):
        return (grad_output * -LAMBDA) # need tune

In [ ]:
loss1_fn = nn.MultiMarginLoss(p=1, margin=MARGIN, weight = None, size_average=True)
loss2_fn = nn.BCELoss()
optimizer1 = torch.optim.Adam(net.parameters(), lr = LEARNING_RATE)
optimizer2 = torch.optim.Adam(net.parameters(), lr = LEARNING_RATE)
#optimizer2 needs a negative learning rate
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for idx,(sample, label) in enumerate(train_loader):
    #ITERATE OVER SAMPLES FROM BOTH DATASETS
    #HOW DO WE GET LABELS FROM THE TARGET DATASET TO ASSESS ACCURACY AND BACK-PROP
    
        sample = Variable(sample, requires_grad = True)
        label = Variable(label, requires_grad = True)
        cosine_sims = sample_and_label_to_cosine_sims(sample, label)
        
        cosine_sims = torch.t(cosine_sims)
        label = torch.squeeze(label)
        
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        
        loss1 = loss1_fn(cosine_sims, label)
        #loss2 = loss2_fn(TODO)
        total_loss = loss1 - LAMBDA * loss2
        
        total_loss.backward()

        optimizer1.step()
        optimizer2.step()
        
        running_loss += total_loss.data[0]
    print "Loss after epoch " + str(epoch) + " :" + str(running_loss)
        

In [13]:
len(train_titles_only)

22853

In [20]:
type(final_train_data_loader)

torch.utils.data.dataset.ConcatDataset

In [21]:
type(train_loader)

NameError: name 'train_loader' is not defined

In [22]:
len(train_titles_only)

22853